# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: bert-base-uncased
* Evaluation approach: transformer trainer
* Fine-tuning dataset: tweet_eval (sentiment)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:

%pip install peft datasets numpy



: 

In [3]:
# optional to set interenet proxy
import os
proxy = ''


os.environ['http_proxy'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [5]:
from datasets import load_dataset
dataset = load_dataset("tweet_eval", "sentiment")
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3) 

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
# Tokenize dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

# Tokenize train and test sets
train_dataset = dataset["train"].map(tokenize, batched=True)
test_dataset = dataset["test"].map(tokenize, batched=True)
validation_dataset = dataset["validation"].map(tokenize, batched=True)


train_dataset_short = train_dataset.remove_columns(["text"]).shuffle(seed=42).select(range(200))
validation_dataset_short = validation_dataset.remove_columns(["text"]).shuffle(seed=42).select(range(40))

train_dataset_medium = train_dataset.remove_columns(["text"]).shuffle(seed=42).select(range(4000))
validation_dataset_medium = validation_dataset.remove_columns(["text"]).shuffle(seed=42).select(range(800))


In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_short,
    eval_dataset=validation_dataset_short,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

c:\DEV\Software\python313\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\A92934909\AppData\Local\Temp\ipykernel_25584\183716395.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.896460,0.625000


TrainOutput(global_step=13, training_loss=1.0166213695819561, metrics={'train_runtime': 232.6029, 'train_samples_per_second': 0.86, 'train_steps_per_second': 0.056, 'total_flos': 52622683545600.0, 'train_loss': 1.0166213695819561, 'epoch': 1.0})

In [74]:
trainer.evaluate()

{'eval_loss': 0.8609987497329712,
 'eval_accuracy': 0.65,
 'eval_runtime': 10.722,
 'eval_samples_per_second': 3.731,
 'eval_steps_per_second': 0.28,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
from peft import get_peft_model, LoraConfig
config = LoraConfig(
    r=16,  
    lora_alpha=16, 
    lora_dropout=0.05,  
    bias="none", 
    target_modules=["query", "value"]  
)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()



trainable params: 589,824 || all params: 110,074,371 || trainable%: 0.5358


In [12]:
from transformers import Trainer, TrainingArguments
import numpy as np


training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_medium,
    eval_dataset=validation_dataset_medium,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

c:\DEV\Software\python313\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\A92934909\AppData\Local\Temp\ipykernel_25584\3930856537.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.996100,0.988813,0.507500


TrainOutput(global_step=500, training_loss=0.9960830078125, metrics={'train_runtime': 3726.0654, 'train_samples_per_second': 1.074, 'train_steps_per_second': 0.134, 'total_flos': 1059701428224000.0, 'train_loss': 0.9960830078125, 'epoch': 1.0})

In [14]:
lora_model.save_pretrained("bert-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [28]:


def get_model_evaluation(model):
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {"accuracy": (predictions == labels).mean()}

    finetuned_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_medium,
        eval_dataset=validation_dataset_medium,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics, 
    )
    
    return finetuned_trainer.evaluate()

In [36]:
print("Bert Base Model Evaluation:")
bert_base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
bert_base_model_evaluation = get_model_evaluation(bert_base_model)
print(bert_base_model_evaluation)

Bert Base Model Evaluation:


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\DEV\Software\python313\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\A92934909\AppData\Local\Temp\ipykernel_25584\3555817882.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  finetuned_trainer = Trainer(


{'eval_loss': 1.140768051147461, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.43, 'eval_runtime': 213.7416, 'eval_samples_per_second': 3.743, 'eval_steps_per_second': 0.468}


In [30]:
from transformers import AutoModelForSequenceClassification

print("Bert Lora Model Evaluation:")
bert_lora_model = AutoModelForSequenceClassification.from_pretrained("bert-lora", num_labels=3)
bert_blora_model_evaluation = get_model_evaluation(bert_lora_model)
print(bert_blora_model_evaluation)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bert Lora Model Evaluation:


C:\Users\A92934909\AppData\Local\Temp\ipykernel_25584\3555817882.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  finetuned_trainer = Trainer(


{'eval_loss': 1.1291580200195312, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.43125, 'eval_runtime': 211.1339, 'eval_samples_per_second': 3.789, 'eval_steps_per_second': 0.474}


In [37]:
all_keys = set(bert_base_model_evaluation.keys()).union(set(bert_blora_model_evaluation.keys()))

print("Training results")

for key in all_keys:
    base_val = bert_base_model_evaluation.get(key, "N/A") 
    bert_val = bert_blora_model_evaluation.get(key, "N/A")
    
    diff = bert_val - base_val
    print(f"{key}: {bert_val=} - {base_val=} = {diff=}" )
     




Training results
eval_model_preparation_time: bert_val=0.0031 - base_val=0.002 = diff=0.0010999999999999998
eval_accuracy: bert_val=0.43125 - base_val=0.43 = diff=0.0012500000000000289
eval_loss: bert_val=1.1291580200195312 - base_val=1.140768051147461 = diff=-0.011610031127929688
eval_runtime: bert_val=211.1339 - base_val=213.7416 = diff=-2.607699999999994
eval_steps_per_second: bert_val=0.474 - base_val=0.468 = diff=0.00599999999999995
eval_samples_per_second: bert_val=3.789 - base_val=3.743 = diff=0.04600000000000026


Training results (after 1 hour) with some data and trainer configuration:

eval_model_preparation_time: bert_val=0.0031 - base_val=0.002 = diff=0.0010999999999999998: Negative
-> The BERT model requires more time for preparation compared to the base model. This could be due to the added complexity and size of the BERT model.

eval_accuracy: bert_val=0.43125 - base_val=0.43 = diff=0.0012500000000000289: Positive
-> The BERT model achieves higher accuracy compared to the base model, indicating it delivers better results on the test data.

eval_loss: bert_val=1.1291580200195312 - base_val=1.140768051147461 = diff=-0.011610031127929688 : Positive
-> The loss value of the BERT model is lower than the base model, suggesting that BERT is better at minimizing errors during predictions.

eval_runtime: bert_val=211.1339 - base_val=213.7416 = diff=-2.607699999999994 : Positive
-> The runtime of the BERT model is slightly faster than the base model, indicating more optimized processing or better execution efficiency.

eval_steps_per_second: bert_val=0.474 - base_val=0.468 = diff=0.00599999999999995 : Positive
-> The number of steps per second is slightly higher in the BERT model, meaning it can perform training or evaluation steps faster.

eval_samples_per_second: bert_val=3.789 - base_val=3.743 = diff=0.04600000000000026 : Positive
-> The BERT model processes more samples per second, further enhancing its efficiency in data processing.
